# Data Exploration
## Introduction
This study presents a comprehensive exploratory data analysis of the Wine Quality datasets, which constitute the empirical foundation for the present investigation. The analysis encompasses both red and white wine variants from the Portuguese "Vinho Verde" collection, with the objective of elucidating the underlying patterns and physicochemical characteristics that determine wine quality assessments.

The analysis commences with the requisite library imports and the configuration of graphical parameters to ensure consistent, publication-standard visualizations throughout the investigative process.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

COLOR_RED = "#720026"
COLOR_WHITE = "#DBDD46"

The first dataset will be loaded and subjected to systematic examination of its constituent features, data types, and structural properties to establish a comprehensive understanding of the data architecture.

In [ ]:
red_wine = pd.read_csv('../data/winequality-red.csv', sep=';')
red_wine.head()

The second dataset will then be examined using the same analytical approach

In [ ]:
white_wine = pd.read_csv('../data/winequality-white.csv', sep=';')
white_wine.head()